# Moving Task-ASL to the Perf Folder for NKI

In this notebook we'll move over the NKI data from task-asl to perf as per BIDS standard and also create the associated aslcontext files.

Note the additional sidecar fields:

```
{
  "PulseSequenceType": "2D", 
  "PulseSequenceDetails" : "SIN" ,
  "LabelingType": "PCASL",
  "LabelingDuration": 1.00,
  "PostLabelingDelay": 1.517,
  "BackgroundSuppression": "Yes",
  "M0":"False",
  "LabelingSlabLocation":"X",
  "LabelingOrientation":"",
  "LabelingDistance":2,
  "AverageLabelingGradient": 34,
  "SliceSelectiveLabelingGradient":45,
  "AverageB1LabelingPulses": 0,
  "LabelingSlabThickness":2,
  "AcquisitionDuration":123,
  "InterPulseSpacing":4,
  "PCASLType":"balanced",
  "PASLType": "",
  "LookLocker":"True",
  "LabelingEfficiency":0.72,
  "BolusCutOffFlag":"False",
  "BolusCutOffTimingSequence":"False",
  "BolusCutOffDelayTime":0,
  "BolusCutOffTechnique":"False"
}

label-control
```

First, get all of the files that could be ASL.

In [89]:
import glob

json_sidecars = []
niftis = []
root_dir = '/cbica/projects/RBC/RBC_RAWDATA/bidsdatasets/NKI/'

for filename in glob.iglob(root_dir + '**/**', recursive=True):
    
    if filename.endswith('.nii.gz') and 'physio' not in filename:
        niftis.append(filename)
    else:
        if 'dataset_description' in filename:
            continue
        elif filename.endswith('.json') and 'physio' not in filename:
            json_sidecars.append(filename)

In [90]:
print(len(niftis))
print(len(json_sidecars))

76408
75612


In [91]:
asl = [x for x in niftis if 'asl' in x.lower()]

In [92]:
len(asl)

6856

In [93]:
all(['task-pCASL' in x for x in asl])

True

In [252]:
import re
import json
import os
import shutil

class ASL:
    
    def __init__(self, full_path):
        
        def parse(fname, regex):
            
            if (match := re.search(regex, fname)) is not None:
                return match.group(0)
            else:
                return None
        
        self.old = full_path
        self.subject = parse(full_path, r'(sub-[a-zA-Z0-9]+)')
        self.session = parse(full_path, r'(ses-[a-zA-Z0-9]+)')
        self.fname = full_path.split('/')[-1]
        self.path = full_path.replace(self.fname, '')
        self.folder = full_path.split('/')[-2]
        self.is_json = False
        self.json = None
        self.is_nifti = False
        self.context = None
        self.is_events = False
        self.is_physio = False
        
        if full_path.endswith('json'):
            
            with open(full_path, 'r') as f:
                self.json = json.load(f)
            self.is_json = True
            
        elif full_path.endswith('events.tsv'):
            self.is_events = True

        elif full_path.endswith('physio.tsv.gz'):
            self.is_physio = True

        else:
            self.is_nifti = True
        
    def convert(self):
        
        new_data = {        
            "PulseSequenceType": "2D", 
            "PulseSequenceDetails" : "SIN" ,
            "LabelingType": "PCASL",
            "LabelingDuration": 1.00,
            "PostLabelingDelay": 1.517,
            "BackgroundSuppression": "Yes",
            "M0":"False",
            "LabelingSlabLocation":"X",
            "LabelingOrientation":"",
            "LabelingDistance":2,
            "AverageLabelingGradient": 34,
            "SliceSelectiveLabelingGradient":45,
            "AverageB1LabelingPulses": 0,
            "LabelingSlabThickness":2,
            "AcquisitionDuration":123,
            "InterPulseSpacing":4,
            "PCASLType":"balanced",
            "PASLType": "",
            "LookLocker":"True",
            "LabelingEfficiency":0.72,
            "BolusCutOffFlag":"False",
            "BolusCutOffTimingSequence":"False",
            "BolusCutOffDelayTime":0,
            "BolusCutOffTechnique":"False"
        }
        
        if self.fname is None:
            raise ValueError
        
        fname = self.fname.replace('_bold', '_asl').replace('_task-pCASL', '')
        
        self.fname = fname
        
        self.folder = 'perf'
        
        self.path = self.path.replace('func', 'perf')
        
        if self.json:
            
            self.json.update(new_data)
            
        if self.is_nifti:
            
            self.context = '/n'.join(['label', 'control'] * 40)
            
    
    def write(self, dry_run=True):
        
        print('Writing data...')
        
        
        if not dry_run:
            
            os.makedirs(self.path, exist_ok = True)
        
        if self.is_json:
            
            
            
            if not dry_run:
                
                print('writing new json file')

                with open(self.full_path(), 'w', encoding='utf-8') as f:
                    json.dump(self.json, f, ensure_ascii=False, indent=4)
            
                print('removing old json file')
                
                os.remove(self.old)
            
                
        
        elif self.is_nifti:
            
            if not dry_run:
                
                print('moving nifti')
                
                shutil.move(self.old, self.full_path())
                #os.remove(self.old)
                
                print('writing context file')
                
                with open(self.full_path().replace('asl.nii.gz', 'aslcontext.tsv'), 'w') as outfile:
                    outfile.write(self.context)
            
        elif self.is_events or self.is_physio:
            
            if not dry_run:
                
                print('moving events file')
                
                shutil.move(self.old, self.full_path())

    def full_path(self):
        
        return self.path + self.fname

    def __str__(self):
        
        return self.path + self.fname

In [223]:
import json
myasl= ASL('/cbica/projects/RBC/RBC_RAWDATA/bidsdatasets/NKI/sub-A00034350/ses-TRT/func/sub-A00034350_ses-TRT_task-pCASL_events.tsv')

In [224]:
print(myasl)
print(myasl.fname)
myasl.convert()
print(myasl.fname)
print(myasl)

/cbica/projects/RBC/RBC_RAWDATA/bidsdatasets/NKI/sub-A00034350/ses-TRT/func/sub-A00034350_ses-TRT_task-pCASL_events.tsv
sub-A00034350_ses-TRT_task-pCASL_events.tsv
sub-A00034350_ses-TRT_events.tsv
/cbica/projects/RBC/RBC_RAWDATA/bidsdatasets/NKI/sub-A00034350/ses-TRT/perf/sub-A00034350_ses-TRT_events.tsv


In [225]:
myasl.write(dry_run=False)

Writing data...
moving events file


Now loop through all

In [238]:
pcasl = []

root_dir = '/cbica/projects/RBC/RBC_RAWDATA/bidsdatasets/NKI/'

for filename in glob.iglob(root_dir + '**/**', recursive=True):
    
    if 'pCASL' in filename and 'physio' not in filename:
        pcasl.append(filename)

In [239]:
pcasl[:20]

['/cbica/projects/RBC/RBC_RAWDATA/bidsdatasets/NKI/sub-A00034350/ses-FLU1/func/sub-A00034350_ses-FLU1_task-pCASL_bold.json',
 '/cbica/projects/RBC/RBC_RAWDATA/bidsdatasets/NKI/sub-A00063003/ses-TRT/func/sub-A00063003_ses-TRT_task-pCASL_bold.json',
 '/cbica/projects/RBC/RBC_RAWDATA/bidsdatasets/NKI/sub-A00063003/ses-BAS1/func/sub-A00063003_ses-BAS1_task-pCASL_bold.json',
 '/cbica/projects/RBC/RBC_RAWDATA/bidsdatasets/NKI/sub-A00063003/ses-FLU1/func/sub-A00063003_ses-FLU1_task-pCASL_bold.json',
 '/cbica/projects/RBC/RBC_RAWDATA/bidsdatasets/NKI/sub-A00052319/ses-BAS1/func/sub-A00052319_ses-BAS1_task-pCASL_bold.json',
 '/cbica/projects/RBC/RBC_RAWDATA/bidsdatasets/NKI/sub-A00074769/ses-BAS1/func/sub-A00074769_ses-BAS1_task-pCASL_bold.json',
 '/cbica/projects/RBC/RBC_RAWDATA/bidsdatasets/NKI/sub-A00074769/ses-FLU1/func/sub-A00074769_ses-FLU1_task-pCASL_bold.json',
 '/cbica/projects/RBC/RBC_RAWDATA/bidsdatasets/NKI/sub-A00075674/ses-BAS1/func/sub-A00075674_ses-BAS1_task-pCASL_bold.json',
 '

In [ ]:
for f in pcasl[:20]:
    
    myasl = ASL(f)
    print(myasl)
    myasl.convert()
    print(myasl)
    
    myasl.write(dry_run=False)

In [250]:
pcasl = []

root_dir = '/cbica/projects/RBC/RBC_RAWDATA/bidsdatasets/NKI/'

for filename in glob.iglob(root_dir + '**/**', recursive=True):
    
    if 'pCASL' in filename and 'physio' not in filename:
        pcasl.append(filename)

In [ ]:
for f in pcasl:
    
    myasl = ASL(f)
    print(myasl)
    myasl.convert()
    print(myasl)
    
    myasl.write(dry_run=False)

In [ ]:
pcasl = []

root_dir = '/cbica/projects/RBC/RBC_RAWDATA/bidsdatasets/NKI/'

for filename in glob.iglob(root_dir + '**/**', recursive=True):
    
    if 'pCASL' in filename and 'physio' not in filename:
        pcasl.append(filename)

In [251]:
len(pcasl)

0

Done!!

Addendum: adding physio files:

In [256]:
pcasl = []

root_dir = '/cbica/projects/RBC/RBC_RAWDATA/bidsdatasets/NKI/'

for filename in glob.iglob(root_dir + '**/**', recursive=True):
    
    if 'pCASL' in filename and 'physio' in filename:
        pcasl.append(filename)

In [ ]:
for f in pcasl:
    
    myasl = ASL(f)
    print(myasl)
    myasl.convert()
    print(myasl)
    
    myasl.write(dry_run=False)

In [257]:
len(pcasl)

0

Addendum: fixing slashes (I wrote them backwards)

In [258]:
pcasl = []

root_dir = '/cbica/projects/RBC/RBC_RAWDATA/bidsdatasets/NKI/'

for filename in glob.iglob(root_dir + '**/**', recursive=True):
    
    if 'aslcontext' in filename:
        pcasl.append(filename)

In [259]:
pcasl[0]

'/cbica/projects/RBC/RBC_RAWDATA/bidsdatasets/NKI/sub-A00034350/ses-TRT/perf/sub-A00034350_ses-TRT_aslcontext.tsv'

In [260]:
with open(pcasl[0], "r") as f:
    data = f.read()

with open(pcasl[0], "w") as f:
    f.write(re.sub(r"/n", r"\n", data))


In [261]:
for fi in pcasl:
    with open(fi, "r") as f:
        data = f.read()

    with open(fi, "w") as f:
        f.write(re.sub(r"/n", r"\n", data))
    